In [1]:
import sys, os
import numpy as np
 
import random
from evoman.environment import Environment
from evoman.controller import Controller
from pygad.kerasga import model_weights_as_vector,model_weights_as_matrix

import tensorflow as tf 

tf.compat.v1.disable_eager_execution()

from deap import base, creator, tools, algorithms

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class player_controller(Controller):

	def __init__(self, model):
		self.model = model

	def control(self, inputs, controller):

		# Normalises the input using min-max scaling
		inputs = (inputs-min(inputs))/float((max(inputs)-min(inputs)))
		inputs = np.array(inputs).reshape((1,20,))

		sol = model_weights_as_matrix(self.model,controller)
		self.model.set_weights(sol)
		preds = self.model.predict(inputs,verbose=0)
		preds = (preds>0.5).astype(int)
		
		return preds[0]

In [3]:
def fitness_function(solution):
    return (env.play(solution)[0],)

In [4]:
input_layer  = tf.keras.layers.Input((20,))
dense_layer1 = tf.keras.layers.Dense(16, activation="relu")(input_layer)
dense_layer2 = tf.keras.layers.Dense(16, activation="relu")(dense_layer1)
output_layer = tf.keras.layers.Dense(5, activation="softmax")(dense_layer2)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [5]:
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
				  player_controller=player_controller(model),
			  	  speed="fastest",
				  enemymode="static",
                  enemies="1",
				  level=2,
                  visuals=True)


MESSAGE: Pygame initialized for simulation.


In [6]:
individual_size = model.count_params()

In [7]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [8]:
toolbox = base.Toolbox()
toolbox.register("weight_bin", np.random.uniform,-2,2)   #Initiate weights from uniform distribution between -1 to +1
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [9]:
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness_function)

In [10]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Min", np.min)

In [14]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)

In [15]:
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.2, ngen=10, halloffame=hof, stats=stats,verbose=True)

gen	nevals	Mean    	Max    	Min     
0  	100   	-4.00158	39.2286	-6.44254
1  	80    	-3.97243	39.0704	-6.42487
2  	84    	-3.24184	93.9373	-6.04025
3  	89    	-3.97484	93.9373	-6.39526
4  	87    	-4.1486 	93.9373	-6.46147
5  	92    	-4.14285	39.0651	-5.98645
6  	86    	-3.42216	39.0651	-5.98645
7  	89    	-3.43465	39.0651	-5.98645


KeyboardInterrupt: 